<h1><center>Exploring Smart Water Analytics</center></h1>

## Aim of this Notebook

The main aim of this notebook is to explore the [Acea Smart water Analytics data](https://www.kaggle.com/c/acea-water-prediction/data) in preliminary sense. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import plotly.express as px
import plotly.graph_objects as go

import missingno as msno

## 1. Aquifers

In this section, we analyze the different aquifer datasets. We have 4 - Doganella, Auser, Luco and Petrignano.  

An aquifer is an underground layer of water-bearing permeable rock or gravel or sand([Source](https://en.wikipedia.org/wiki/Aquifer)).

![](https://i.pinimg.com/originals/8d/aa/da/8daada2ad12924b9f8d84fab45a3c754.jpg)
<center><a href="https://i.pinimg.com/originals/8d/aa/da/8daada2ad12924b9f8d84fab45a3c754.jpg">Source</a></center>

In [ ]:
aq_d = pd.read_csv("/kaggle/input/acea-water-prediction/Aquifer_Doganella.csv")
aq_a = pd.read_csv("/kaggle/input/acea-water-prediction/Aquifer_Auser.csv")
aq_p = pd.read_csv("/kaggle/input/acea-water-prediction/Aquifer_Luco.csv")
aq_l = pd.read_csv("/kaggle/input/acea-water-prediction/Aquifer_Petrignano.csv")

## Aquifer Doganella

- Fed by 2 underground aquifers
    - Upper stratum is a water table of 30m thickness
    - Semi-confined artesian aquifer with a 50m thickness
- Has 9 outputs of the format `Depth_to_Groundwater_Pozzo_{n}`, where n is between [1,9]
- The aquifers are assessed through 9 wells
- ~80% of drainage comes from the artesian aquifer

**Features Description**  

| Field | Format | Description |  
| ----- | ------ | ----------- |
| Date | Date | Primary Key |
| Rainfall_X | Real number | Qty. of rain(mm) in area X |
| Depth_to_Groundwater_Pozzo_Y | Real number | Groundwater level(m) from ground, detected by piezometer Y | 
| Temperature_Z | Real Number | Temp(degree celsius), measured by thermometric station Z |
| Volume_Pozzo_K | Real Number | Vol of water(mc), taken from drinking water treatment plan K |

In [ ]:
def get_day(x):
    
    return x.strftime('%d')

def get_month(x):
    
    return x.strftime('%m')

def get_month_name(x):
    
    return x.strftime('%B')

def get_year(x):
    
    return x.strftime('%Y')

aq_d['Date'] = pd.to_datetime(aq_d['Date'], format = "%d/%m/%Y")
aq_d['Day'] = aq_d['Date'].apply(get_day)
aq_d['Month'] = aq_d['Date'].apply(get_month)
aq_d['Month_name'] = aq_d['Date'].apply(get_month_name)
aq_d['Year'] = aq_d['Date'].apply(get_year)

In [ ]:
# interactive viz functions

def plot_complete_trend(df, var):
    
    fig = go.Figure([
        go.Scatter(
            x = df['Date'],
            y = df[var],
            line = dict(color = '#000080'))
    ])
    
    return fig

def plot_yr_avg(df, var):
    
    yearly_avg = df.groupby(['Year']).agg({var: 'mean'})
    fig = px.line(yearly_avg)
    
    return fig
    
def plot_mon_avg(df, var):
    
    monthly_avg = df.groupby(['Month']).agg({var: 'mean'})
    fig = px.line(monthly_avg)
    
    return fig

### Missing Value Analysis

It is evident that the Aquifer Doganella dataset has quite a few null values. In this section, I try to ananlyse these missing values. In order to do this, the [missingno package](https://github.com/ResidentMario/missingno) is resourceful.

In [ ]:
msno.matrix(aq_d, color = (0, 0, 0.52))
plt.show()

**🔎 How to interpret the above plot**  
- Each bar represents a feature
    - The whitespaces represent missing values
    - Read it top-down i.e, the highest point is the first observation
- The spikes on the right depict observation-wise(row-wise) missingness

**💡 Insights**  
- The features that are concerned with `Depth_to_Groundwater_Y` and `Volume_Pozzo_K` have the most missing values
    - It does seem as though there is a relation between the missingness of these features
- The `Temperature_Z` features seem to have missingness together towards the final observations
- The `Volume_Pozzo_K` features show an interesting pattern
    - Though they have very few missing values, it is to be noticed that almost all values for all features are non-missing after a particular date
        - This might imply that measurement methods were only put into effect from this date onwards

In [ ]:
# check nullity correlation
msno.heatmap(aq_d)
plt.show()

**🔎 How to interpret the above plot**  
- The plot above is a nullity correlation heatmap
    - Nullity correlation refers to how strongly the presence or absence of a variable influences the presence or absence of another
    - A value of 1 indicates that if one is present, other is also present
    - A value of -1 indicates that if one is present, other is absent
    - A value of 0 indictaes no correlation

**💡 Insights**  
- All the `Volume_Pozzo_K` features have a nullity correlation of 1 with each other; which re-inforces the previous idea we had that measurements only would have begun at a particular date
- The`Temperature_Z` features have a strong positive nullity correlation of 0.9
- The `Depth_to_Groundwater_Pozzo_Y` features have strong positive nullity correlations with each other

### Groundwater Level Trends

In [ ]:
temp = aq_d.set_index('Date')

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('Depth to Groundwater(m) in the 9 Wells')

sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_1'], ax=axes[0, 0], color = '#000080')
axes[0, 0].set_title('Well 1')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_2'], ax=axes[0, 1], color = '#000080')
axes[0, 1].set_title('Well 2')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_3'], ax=axes[0, 2], color = '#000080')
axes[0, 2].set_title('Well 3')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_4'], ax=axes[1, 0], color = '#000080')
axes[1, 0].set_title('Well 4')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_5'], ax=axes[1, 1], color = '#000080')
axes[1, 1].set_title('Well 5')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_6'], ax=axes[1, 2], color = '#000080')
axes[1, 2].set_title('Well 6')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_7'], ax=axes[2, 0], color = '#000080')
axes[2, 0].set_title('Well 7')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_8'], ax=axes[2, 1], color = '#000080')
axes[2, 1].set_title('Well 8')
sns.lineplot(data = temp['Depth_to_Groundwater_Pozzo_9'], ax=axes[2, 2], color = '#000080')
axes[2, 2].set_title('Well 9')

plt.show()

**💡 Insights**  

- Each of the 9 wells have a separate pattern among the last 8 years
- Well 1 has the highest level of groundwater when compared to all other wells over the years
- Well 3 has the lowest level of groundwater when compared to all other wells over the years
- Wells where the level of groundwater has been generally decreasing include 1, 2, 4, 6 and 8
- Wells where the levels have been generally increasing include 3 and 9

### Volume of water in the wells

In [ ]:
temp = aq_d.set_index('Date')

fig, axes = plt.subplots(4, 2, figsize=(15, 15))
fig.suptitle('Volume of water(m) from drinking water treatment plants')

sns.lineplot(data = temp['Volume_Pozzo_1'], ax=axes[0, 0], color = '#000080')
axes[0, 0].set_title('Well 1')
sns.lineplot(data = temp['Volume_Pozzo_2'], ax=axes[0, 1], color = '#000080')
axes[0, 1].set_title('Well 2')
sns.lineplot(data = temp['Volume_Pozzo_3'], ax=axes[1, 0], color = '#000080')
axes[1, 0].set_title('Well 3')
sns.lineplot(data = temp['Volume_Pozzo_4'], ax=axes[1, 1], color = '#000080')
axes[1, 1].set_title('Well 4')
sns.lineplot(data = temp['Volume_Pozzo_5+6'], ax=axes[2, 0], color = '#000080')
axes[2, 0].set_title('Well 5+6')
sns.lineplot(data = temp['Volume_Pozzo_7'], ax=axes[2, 1], color = '#000080')
axes[2, 1].set_title('Well 7')
sns.lineplot(data = temp['Volume_Pozzo_8'], ax=axes[3, 0], color = '#000080')
axes[3, 0].set_title('Well 8')
sns.lineplot(data = temp['Volume_Pozzo_9'], ax=axes[3, 1], color = '#000080')
axes[3, 1].set_title('Well 9')

plt.show()

**💡 Insights**  

- Well 1 shows the most changes in volume
    - Could be attributed to the fact its groundwater level is higher than other wells and easily accessible
- Volume in well 3 has been decreasing in recent times

**❓ Questions**

- It's not yet clear at this point why 5 and 6 are measured together

### Temperatures in Monteporzio and Velletri

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 9))
fig.suptitle('Temperature(degree celsius) in both areas')

months = ["January", "February", "March", "April", "May", "June", "July", "August",
         "September", "October", "November", "December"]

temp = aq_d.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Monteporzio': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Monteporzio'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[0])
axes[0].set_title('Monteporzio')

temp = aq_d.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Velletri': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Velletri'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[1])
axes[1].set_title('Velletri')

plt.show()

**🔎 How to interpret the above plot** 

- Each cell represent the average temperature of a region for that month in that year
- Towards red => Warmer

**💡 Insights**  

- The temperature patterns of both regions are very similar, that is expected
- Temperatures in Monteporzio are lower than those in Velletri

### Rainfall in Monteporzio and Velletri

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 9))
fig.suptitle('Rainfall(mm) in both areas')

months = ["January", "February", "March", "April", "May", "June", "July", "August",
         "September", "October", "November", "December"]

temp = aq_d.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Monteporzio': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Monteporzio'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[0])
axes[0].set_title('Monteporzio')

temp = aq_d.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Velletri': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Velletri'),
            annot=True, linewidths=.5, cmap="Blues", ax = axes[1])
axes[1].set_title('Velletri')

plt.show()

**🔎 How to interpret the above plot** 

- Each cell represent the average rainfall for a region for that month in that year
- Towards red => Warmer

**💡 Insights**  

- At this point, it's hard to find something here

## Aquifer Auser

- Consists of 2 subsystems, NORTH and SOUTH
    - NORTH partly influences the behaviour of SOUTH
    - NORTH is an unconfined aquifer
    - SOUTH is a confined aquifer
- Levels of NORTH represented by SAL, PAG, CoS and DIEC wells
- Levels of SOUTH represented by LT2
- Has 5 outputs
    - Depth_to_Groundwater_SAL
    - Depth_to_Groundwater_CoS
    - Depth_to_Groundwater_LT2
    - Depth_to_Groundwater_PAG
    - Depth_to_Groundwater_DIEC

**Features Description**  

| Field | Format | Description |  
| ----- | ------ | ----------- |
| Date | Date | Primary Key |
| Rainfall_X | Real number | Qty. of rain(mm) in area X |
| Depth_to_Groundwater_Y | Real number | Groundwater level(m) from ground, detected by piezometer Y | 
| Temperature_Z | Real Number | Temp(degree celsius), measured by thermometric station Z |
| Volume_H | Real Number | Vol of water(mc), taken from drinking water treatment plan H |
| Hydrometry_K | Real Number | Groundwater level(m) detected by hydrometric station K |

In [ ]:
aq_a['Date'] = pd.to_datetime(aq_a['Date'], format = "%d/%m/%Y")
aq_a['Day'] = aq_a['Date'].apply(get_day)
aq_a['Month'] = aq_a['Date'].apply(get_month)
aq_a['Month_name'] = aq_a['Date'].apply(get_month_name)
aq_a['Year'] = aq_a['Date'].apply(get_year)

### Missing Value Analysis

In [ ]:
msno.matrix(aq_a, color = (0, 0, 0.52))
plt.show()

In [ ]:
# check nullity correlation
msno.heatmap(aq_a)
plt.show()

**💡 Insights**  

- The nullity correlations between the `Volume_H`, `Depth_to_Groundwater_Y` and `Rainfall_X` are strongly positive

### Groundwater Level Trends

In [ ]:
temp = aq_a.set_index('Date')

fig, axes = plt.subplots(3, 2, figsize=(15, 15))
fig.suptitle('Depth to Groundwater(m) in the 5 Wells')

sns.lineplot(data = temp['Depth_to_Groundwater_LT2'], ax=axes[0, 0], color = '#000080')
axes[0, 0].set_title('Well LT2')
sns.lineplot(data = temp['Depth_to_Groundwater_SAL'], ax=axes[0, 1], color = '#000080')
axes[0, 1].set_title('Well SAL')
sns.lineplot(data = temp['Depth_to_Groundwater_PAG'], ax=axes[1, 0], color = '#000080')
axes[1, 0].set_title('Well PAG')
sns.lineplot(data = temp['Depth_to_Groundwater_CoS'], ax=axes[1, 1], color = '#000080')
axes[1, 1].set_title('Well CoS')
sns.lineplot(data = temp['Depth_to_Groundwater_DIEC'], ax=axes[2, 0], color = '#000080')
axes[2, 0].set_title('Well DIEC')
axes[-1,-1].axis('off')

plt.show()

**💡 Insights**  

- The wells where the depth to groundwater has been decreasing(favourable condition) over the years are LT2 and SAL
    - There could be an *interesting explanation* to this. Read ahead.
- Yet, all wells show several trend changes over the past years
- In terms of when measurement began, the following is the order of the wells (starting with the oldest)
    - LT2, CoS > SAL > PAG > DIET
- While LT2 and SAL have had relatively better improvement in increased ground water levels, it is to be noted that these 2 wells still have much lower ground water levels(especially LT2 where there is a clear improvement trend) than the other wells.
    - So, probably drastic interventions were put in place to improve groundwater levels in LT2
    - Or maybe I am just being too ambitious here :)

### Volume of water in wells

In [ ]:
temp = aq_a.set_index('Date')

fig, axes = plt.subplots(3, 2, figsize=(15, 15))
fig.suptitle('Volume of water(m) from drinking water treatment plants')

sns.lineplot(data = temp['Volume_POL'], ax=axes[0, 0], color = '#000080')
axes[0, 0].set_title('POL')
sns.lineplot(data = temp['Volume_CC1'], ax=axes[0, 1], color = '#000080')
axes[0, 1].set_title('CC1')
sns.lineplot(data = temp['Volume_CC2'], ax=axes[1, 0], color = '#000080')
axes[1, 0].set_title('CC2')
sns.lineplot(data = temp['Volume_CSA'], ax=axes[1, 1], color = '#000080')
axes[1, 1].set_title('CSA')
sns.lineplot(data = temp['Volume_CSAL'], ax=axes[2, 0], color = '#000080')
axes[2, 0].set_title('CSAL')
axes[-1,-1].axis('off')

plt.show()

**💡 Insights**  

- If the negatives confuse you, don't worry. It's because the volume is the volume taken from the pumps and sent to the water treatment plant ([Source](https://www.kaggle.com/c/acea-water-prediction/discussion/202675)).
- CSA and CSAL began doing this only in late 2019

## Temperatures

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('Temperature(degree celsius)')

months = ["January", "February", "March", "April", "May", "June", "July", "August",
         "September", "October", "November", "December"]

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Orentano': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Orentano'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[0,0])
axes[0,0].set_title('Orentano')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Ponte_a_Moriano': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Ponte_a_Moriano'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[0,1])
axes[0,1].set_title('Ponte_a_Moriano')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Monte_Serra': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Monte_Serra'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[1,0])
axes[1,0].set_title('Monte_Serra')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Temperature_Lucca_Orto_Botanico': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Temperature_Lucca_Orto_Botanico'),
            annot=True, linewidths=.5, cmap="bwr", ax = axes[1,1])
axes[1,1].set_title('Lucca_Orto_Botanico')

plt.show()

## Rainfall

In [ ]:
fig, axes = plt.subplots(5, 2, figsize=(20, 35))
fig.suptitle('Rainfall(mm)')

months = ["January", "February", "March", "April", "May", "June", "July", "August",
         "September", "October", "November", "December"]

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Gallicano': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Gallicano'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[0,0])
axes[0,0].set_title('Gallicano')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Pontetetto': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Pontetetto'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[0,1])
axes[0,1].set_title('Pontetetto')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Monte_Serra': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Monte_Serra'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[1,0])
axes[1,0].set_title('Monte_Serra')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Orentano': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Orentano'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[1,1])
axes[1,1].set_title('Orentano')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Borgo_a_Mozzano': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Borgo_a_Mozzano'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[2,0])
axes[2,0].set_title('Borgo_a_Mozzano')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Fabbriche_di_Vallico': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Fabbriche_di_Vallico'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[2,1])
axes[2,1].set_title('Fabbriche_di_Vallico')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Piaggione': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Piaggione'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[3,0])
axes[3,0].set_title('Piaggione')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Calavorno': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Calavorno'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[3,1])
axes[3,1].set_title('Calavorno')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Croce_Arcana': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Croce_Arcana'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[4,0])
axes[4,0].set_title('Croce_Arcana')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Rainfall_Tereglio_Coreglia_Antelminelli': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Rainfall_Tereglio_Coreglia_Antelminelli'),
            annot=True, fmt = '.2g', linewidths=.5, cmap="Blues", ax = axes[4,1])
axes[4,1].set_title('Tereglio_Coreglia_Antelminelli')


plt.show()

In [ ]:
"""
# Hydrometry groundwater level(m)
# what are bothe pos and neg values doing?

fig, axes = plt.subplots(1, 2, figsize=(25, 9))
fig.suptitle('Hydrometry Groundwater Level(m)')

months = ["January", "February", "March", "April", "May", "June", "July", "August",
         "September", "October", "November", "December"]

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Hydrometry_Monte_S_Quirico': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Hydrometry_Monte_S_Quirico'),
            annot=True, fmt = '.1g', linewidths=.5, cmap="Blues", ax = axes[0])
axes[0].set_title('Monte_S_Quirico')

temp = aq_a.groupby(['Month_name', 'Year'], as_index=False).agg({'Hydrometry_Piaggione': 'mean'})
temp['Month_name'] = pd.Categorical(temp['Month_name'], categories=months, ordered=True)
sns.heatmap(temp.pivot('Month_name', 'Year', 'Hydrometry_Piaggione'),
            annot=True, fmt = '.1g', linewidths=.5, cmap="Blues", ax = axes[1])
axes[1].set_title('Piaggione')

plt.show()
"""

🚧 Under construction. Will update this notebook with further work :)

## References

1. https://dev.to/thalesbruno/subplotting-with-matplotlib-and-seaborn-5ei8
2. https://thetechbuddies.com/2019/06/25/annotated-heatmap-visualization-for-weather-data-using-pandas-and-seaborn/